In [1]:
import pandas as pd, numpy as np
import _pickle as pkl
import math

from sklearn.model_selection import train_test_split

In [2]:
# df_movies = pd.read_csv('./input/SKB_DLP_MOVIES.csv', encoding='cp949')
# df_test = pd.read_csv('./input/SKB_DLP_QUESTION.csv', encoding='cp949')
df_views = pd.read_csv('./input/SKB_DLP_VIEWS.csv', encoding='cp949')

In [3]:
# build user set 
users = df_views.USER_ID.unique()

# user별 data 체크
userlen = len(users)
uid_stan, idx_stan = 0, 0
start_idx, end_idx = 0, 0

df_meta = []
for i, uid in enumerate(df_views.USER_ID):
    if i % 10000 ==0:
        print("\r{}/{}".format(uid, userlen), end='')
    if uid != uid_stan:
        end_idx = i-1
        df_meta.append(np.array([uid-1, i- idx_stan, start_idx, end_idx]))
        start_idx = i
        idx_stan = i
        uid_stan = uid
        
    else:
        continue
end_idx = i-1
df_meta.append(np.array([uid-1, i- idx_stan, start_idx, end_idx]))
df_meta = np.array(df_meta)

227023/227200

# Train / Valid Split

In [4]:
# filter small user
small_user = df_meta[df_meta[:,1]<11][:,0]
users = np.delete(users, small_user)

np.random.seed(seed=1991)
np.random.shuffle(users)

fold_size = math.ceil(len(users)/4)

In [7]:
stride = 'hexa5'

In [9]:
for k in range(4):
    print(k)
    valid_user = users[k*fold_size : (k+1)*fold_size]
    train_user = users[np.invert(np.isin(users, valid_user))]

    train_meta = df_meta[np.isin(df_meta[:, 0], train_user)]
    valid_meta = df_meta[np.isin(df_meta[:, 0], valid_user)]
    # user별 Data Slicing
    userlen = len(users)

    valids_x, valids_y = [], []
    # valid preprcoess // 10개까진 train용, 11부터는 label용
    for uid, length, start, end in valid_meta:
        if uid % 8000 == 0:
            print("\r{}/{}".format(uid, len(df_meta)), end='')

        subset = df_views.iloc[start:end+1]
        data = subset[:10].values.tolist()
        label = subset[10:].MOVIE_ID.unique().tolist()
        valids_x.append(data)
        valids_y.append(np.array(label, dtype=np.uint16))    

    trains_x, trains_y = [], []
    
    # train preprocess
    for uid, length, start, end in train_meta:
        if uid % 8000 == 0:
            print("\r{}/{}".format(uid, len(df_meta)), end='')

        subset = df_views.iloc[start:end+1]
        for i in range(0, 26, 5):
    
            data = subset[i:i+10].values.tolist()
            label = subset[i+10:].MOVIE_ID.unique().tolist()
            if label != []:
                trains_x.append(data)
                trains_y.append(np.array(label, dtype=np.uint16))

    with open('./temp/train_slide_last_{}_{}.pkl'.format(stride, k), 'wb') as f:
        pkl.dump([trains_x, trains_y], f)
    with open('./temp/valid_slide_last_{}_{}.pkl'.format(stride, k), 'wb') as f:
        pkl.dump([valids_x, valids_y], f)
    print()

3
216000/227200

# End of Preprocessing